### Calculating annual mean value of self-calibrating Palmer Drought Severity Index (scPDSI) tif files
https://crudata.uea.ac.uk/cru/data/drought/#global

In [15]:
from osgeo import gdal, osr
import os
import numpy as np
import dask.array as da
import time
import dask
import imageio
import glob


In [4]:
folder_list=glob.glob('../data/drought/*/')
folder_list=sorted(folder_list)

In [6]:
yearly_imgs_list = [sorted(glob.glob(folder+'*.tif')) for folder in folder_list]

In [23]:
%%time
for index,imgs in enumerate(yearly_imgs_list):
    image_status=False
    counter=0
    while image_status is False:
        try:            
            output_folder='../data/drought/mean_ndvi/'              
                
            target_imgs=[item for item in imgs]
            
            array_list=[]
            for index,img in enumerate(target_imgs):
                ds = gdal.Open(img)
                array=ds.ReadAsArray()
                dask_array=da.from_array(array,chunks=len(array)/2)
                array_list.append(dask_array)
            
            stack = da.stack(array_list, axis=0)
            max_array=stack.mean(axis=0).compute()
            
            s_srs = ds.GetProjectionRef()    
            osng = osr.SpatialReference ()
                       
            #use last img to define name of the last product
            folder_name=img.split('/')[-2]
            
            osng.SetFromUserInput ( s_srs )    
            geo_t = ds.GetGeoTransform ()    
            x_size = ds.RasterXSize # Raster xsize
            y_size = ds.RasterYSize # Raster ysize
            mem_drv= gdal.GetDriverByName( 'MEM')
            dest = mem_drv.Create('', x_size,y_size, 1)
            dest.SetGeoTransform( geo_t )
            dest.SetProjection( osng.ExportToWkt())
            dest.GetRasterBand(1).WriteArray(max_array)
            output_max_path=output_folder+'Drought_scPDSI_Mean_'+folder_name+'.tif'
            gdal.Warp(output_max_path, dest, format = 'GTiff')
            max_array=None
            dest=None
            ds=None
            mem_drv=None
            image_status=True
            #print(output_max_path)
            
        except:            
            image_status=False
            counter+=1
            if counter==10:
                print('couldnt be calculated')
                break
            continue
    

CPU times: user 4.07 s, sys: 549 ms, total: 4.62 s
Wall time: 4.26 s
